# Loading

In [ ]:
!pip install -U -q "google-genai"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.9/165.9 kB 4.8 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import copy
import string
import re
import json
import random
import time

from google import genai
from google.genai import types

In [ ]:
client = genai.Client(api_key="your-api-key-here")

In [ ]:
for model_info in client.models.list():
    print(model_info.name)

models/chat-bison-001
models/text-bison-001
models/embedding-gecko-001
models/gemini-1.0-pro-vision-latest
models/gemini-pro-vision
models/gemini-1.5-pro-latest
models/gemini-1.5-pro-001
models/gemini-1.5-pro-002
models/gemini-1.5-pro
models/gemini-1.5-flash-latest
models/gemini-1.5-flash-001
models/gemini-1.5-flash-001-tuning
models/gemini-1.5-flash
models/gemini-1.5-flash-002
models/gemini-1.5-flash-8b
models/gemini-1.5-flash-8b-001
models/gemini-1.5-flash-8b-latest
models/gemini-1.5-flash-8b-exp-0827
models/gemini-1.5-flash-8b-exp-0924
models/gemini-2.5-pro-exp-03-25
models/gemini-2.5-pro-preview-03-25
models/gemini-2.5-flash-preview-04-17
models/gemini-2.0-flash-exp
models/gemini-2.0-flash
models/gemini-2.0-flash-001
models/gemini-2.0-flash-exp-image-generation
models/gemini-2.0-flash-lite-001
models/gemini-2.0-flash-lite
models/gemini-2.0-flash-lite-preview-02-05
models/gemini-2.0-flash-lite-preview
models/gemini-2.0-pro-exp
models/gemini-2.0-pro-exp-02-05
models/gemini-exp-1206
m

In [ ]:
for model_info in client.tunings.list():
    print(model_info.name)

tunedModels/salarytunedmodel1-o7ojl7z82wbp
tunedModels/salarytunedmodel1-8m0fccs9srwj
tunedModels/salarytunedmodel1-636kmk97vwof
tunedModels/salarytunedmodel2-3h17mzl216ez
tunedModels/salarytunedmodel3-e6mptc3xnfii


# Dataset Prep

In [ ]:
train_df = pd.read_csv('data/salary_labelled_development_set_cleaned.csv')
test_df = pd.read_csv('data/salary_labelled_test_set_cleaned.csv')

In [ ]:
# df to store model predictions
train_format_df = pd.DataFrame(columns=["text_input", "output"])

In [ ]:
test_format_df = pd.DataFrame(columns=["text_input", "output"])

In [ ]:
for i in range(len(train_df)):
# for i in range(200):

  desc = f"""job_title: {train_df.iloc[i].job_title},
nation_short_desc: {train_df.iloc[i].nation_short_desc},
salary_additional_text: {train_df.iloc[i].salary_additional_text},
job_ad_details: {train_df.iloc[i].job_ad_details},
"""

  desc += """
Extract structured salary information from this job description in the format: min-max-currency-frequency.
Always abide by the range in 'salary_additional_text:', if it is provided.
Respond in JSON: {"MinSalary": "", "MaxSalary": "", "Currency": "", "Frequency": ""}.
If not provided explicitly, output 0 for "MinSalary" and "MaxSalary", and "None" for "Currency" and "Frequency".
If the salary is mentioned, always output a range, where MinSalary and MaxSalary can be equal.
Use 'nation_short_desc:' to determine the correct currency.
Output the currency as 3 letters. Use adverb to output frequency (annual, monthly, daily or hourly).
"""

  min_salary, max_salary, currency, frequency = train_df.iloc[i].y_true.split('-')
  label = f'{{"MinSalary": "{min_salary}", "MaxSalary": "{max_salary}", "Currency": "{currency}", "Frequency": "{frequency}"}}'

  train_format_df.loc[i] = [desc, label]

In [ ]:
train_format_df

,text_input,output
0,job_title: Financial Account - Call Center Age...,"{""MinSalary"": ""17500"", ""MaxSalary"": ""17500"", ""..."
1,job_title: Aspiring Call Center Agents - Work ...,"{""MinSalary"": ""16000"", ""MaxSalary"": ""16000"", ""..."
2,job_title: Production Staff Required - Afterno...,"{""MinSalary"": ""0"", ""MaxSalary"": ""0"", ""Currency..."
3,"job_title: Payer Analyst,\nnation_short_desc: ...","{""MinSalary"": ""0"", ""MaxSalary"": ""0"", ""Currency..."
4,"job_title: Solicitor, Restructuring ,\nnatio...","{""MinSalary"": ""0"", ""MaxSalary"": ""0"", ""Currency..."
...,...,...
2248,"job_title: Senior Manager, Process Excellence ...","{""MinSalary"": ""0"", ""MaxSalary"": ""0"", ""Currency..."
2249,job_title: Accounting Manager - Hotel Business...,"{""MinSalary"": ""0"", ""MaxSalary"": ""0"", ""Currency..."
2250,"job_title: Technical Operator - Audio Visual,\...","{""MinSalary"": ""0"", ""MaxSalary"": ""0"", ""Currency..."
2251,"job_title: Customer Service Officer ,\nnation_...","{""MinSalary"": ""2500"", ""MaxSalary"": ""3000"", ""Cu..."


In [ ]:
for i in range(len(test_df)):

  desc = f"""job_title: {test_df.iloc[i].job_title},
nation_short_desc: {test_df.iloc[i].nation_short_desc},
salary_additional_text: {test_df.iloc[i].salary_additional_text},
job_ad_details: {test_df.iloc[i].job_ad_details},
"""

  desc += """
Extract structured salary information from this job description in the format: min-max-currency-frequency.
Always abide by the range in 'salary_additional_text:', if it is provided.
Respond in JSON: {"MinSalary": "", "MaxSalary": "", "Currency": "", "Frequency": ""}.
If not provided explicitly, output 0 for "MinSalary" and "MaxSalary", and "None" for "Currency" and "Frequency".
If the salary is mentioned, always output a range, where MinSalary and MaxSalary can be equal.
Use 'nation_short_desc:' to determine the correct currency.
Output the currency as 3 letters. Use adverb to output frequency (annual, monthly, daily or hourly).
"""

  label = str(test_df.iloc[i].y_true)

  test_format_df.loc[i] = [desc, label]

In [ ]:
test_format_df

,text_input,output
0,"job_title: Cashier ,\nnation_short_desc: MY,\n...",1500-1800-MYR-MONTHLY
1,"job_title: 學校派飯員,\nnation_short_desc: HK,\nsal...",60-60-HKD-HOURLY
2,"job_title: Skid Steer Operator/ Truck Driver ,...",0-0-None-None
3,"job_title: Business Development Manager,\nnati...",0-0-None-None
4,"job_title: Workplace and Safety Officer,\nnati...",0-0-None-None
...,...,...
562,job_title: Trainee Rail Safety Officer - Taura...,26-26-NZD-HOURLY
563,job_title: Procurement Manager - Logistics Com...,0-0-None-None
564,job_title: Process Operators - Annies Fruit Ba...,0-0-None-None
565,job_title: HONG LEONG BANK RECOVERY OFFICER / ...,1500-2500-MYR-MONTHLY


# Tuning

In [ ]:
# create tuning model
training_dataset=types.TuningDataset(
        examples=[
            types.TuningExample(
                text_input=train_format_df.iloc[i].text_input,
                output=train_format_df.iloc[i].output,
            )
            for i in range(len(train_format_df))
        ],
    )

In [ ]:
tuning_job = client.tunings.tune(
    base_model='models/gemini-1.5-flash-001-tuning',
    training_dataset=training_dataset,
    # hyperparameters
    config=types.CreateTuningJobConfig(
        epoch_count= 5,
        batch_size=4,
        learning_rate=0.001,
        tuned_model_display_name="salary_tuned_model_3"
    )
)

name='tunedModels/salarytunedmodel3-e6mptc3xnfii' state=<JobState.JOB_STATE_QUEUED: 'JOB_STATE_QUEUED'> create_time=None start_time=None end_time=None update_time=None error=None description=None base_model=None tuned_model=None supervised_tuning_spec=None tuning_data_stats=None encryption_spec=None partner_model_tuning_spec=None distillation_spec=None experiment=None labels=None pipeline_job=None tuned_model_display_name=None


In [ ]:
tuning_job = client.tunings.get(name=tuning_job.name)
print(tuning_job)

name='tunedModels/salarytunedmodel3-e6mptc3xnfii' state=<JobState.JOB_STATE_RUNNING: 'JOB_STATE_RUNNING'> create_time=datetime.datetime(2025, 4, 22, 16, 51, 50, 152371, tzinfo=TzInfo(UTC)) start_time=datetime.datetime(2025, 4, 22, 16, 51, 50, 640322, tzinfo=TzInfo(UTC)) end_time=None update_time=datetime.datetime(2025, 4, 22, 16, 51, 50, 152371, tzinfo=TzInfo(UTC)) error=None description=None base_model='models/gemini-1.5-flash-001-tuning' tuned_model=TunedModel(model='tunedModels/salarytunedmodel3-e6mptc3xnfii', endpoint='tunedModels/salarytunedmodel3-e6mptc3xnfii') supervised_tuning_spec=None tuning_data_stats=None encryption_spec=None partner_model_tuning_spec=None distillation_spec=None experiment=None labels=None pipeline_job=None tuned_model_display_name=None


In [ ]:
tuning_job

TuningJob(name='tunedModels/salarytunedmodel3-e6mptc3xnfii', state=<JobState.JOB_STATE_RUNNING: 'JOB_STATE_RUNNING'>, create_time=datetime.datetime(2025, 4, 22, 16, 51, 50, 152371, tzinfo=TzInfo(UTC)), start_time=datetime.datetime(2025, 4, 22, 16, 51, 50, 640322, tzinfo=TzInfo(UTC)), end_time=None, update_time=datetime.datetime(2025, 4, 22, 16, 51, 50, 152371, tzinfo=TzInfo(UTC)), error=None, description=None, base_model='models/gemini-1.5-flash-001-tuning', tuned_model=TunedModel(model='tunedModels/salarytunedmodel3-e6mptc3xnfii', endpoint='tunedModels/salarytunedmodel3-e6mptc3xnfii'), supervised_tuning_spec=None, tuning_data_stats=None, encryption_spec=None, partner_model_tuning_spec=None, distillation_spec=None, experiment=None, labels=None, pipeline_job=None, tuned_model_display_name=None)

# Testing

In [ ]:
tuning_job = client.tunings.get(name='tunedModels/salarytunedmodel3-e6mptc3xnfii')
print(tuning_job)

name='tunedModels/salarytunedmodel3-e6mptc3xnfii' state=<JobState.JOB_STATE_SUCCEEDED: 'JOB_STATE_SUCCEEDED'> create_time=datetime.datetime(2025, 4, 22, 16, 51, 50, 152371, tzinfo=TzInfo(UTC)) start_time=datetime.datetime(2025, 4, 22, 16, 51, 50, 640322, tzinfo=TzInfo(UTC)) end_time=datetime.datetime(2025, 4, 22, 18, 38, 3, 324561, tzinfo=TzInfo(UTC)) update_time=datetime.datetime(2025, 4, 22, 18, 38, 3, 324561, tzinfo=TzInfo(UTC)) error=None description=None base_model='models/gemini-1.5-flash-001-tuning' tuned_model=TunedModel(model='tunedModels/salarytunedmodel3-e6mptc3xnfii', endpoint='tunedModels/salarytunedmodel3-e6mptc3xnfii') supervised_tuning_spec=None tuning_data_stats=None encryption_spec=None partner_model_tuning_spec=None distillation_spec=None experiment=None labels=None pipeline_job=None tuned_model_display_name=None


## Qualitative

In [ ]:
test_index = 174

response = client.models.generate_content(
      model='tunedModels/salarytunedmodel3-e6mptc3xnfii',
      contents=test_format_df.iloc[test_index].text_input,
  )

response.text

'{"MinSalary": "25000", "MaxSalary": "35000", "Currency": "THB", "Frequency": "MONTHLY"}'

## Quantitative

In [ ]:
# create df to capture results
results_df = pd.DataFrame(columns=["y_true", "y_pred"])

In [1]:
for i in range(len(test_format_df)):
  while True:
    try:
      response = client.models.generate_content(
        model='tunedModels/salarytunedmodel3-e6mptc3xnfii',
        contents=test_format_df.iloc[i].text_input,
      )
      results_df.loc[len(results_df)] = [test_format_df.iloc[i].output, response.text]
      break  # break the retry loop if successful

    except Exception as e:
      retry_delay = 60  # default delay

      print(f"Waiting {retry_delay}s to retry at index {i}")
      time.sleep(retry_delay)

Waiting 60s to retry at index 15
Waiting 60s to retry at index 31
Waiting 60s to retry at index 47
Waiting 60s to retry at index 63
Waiting 60s to retry at index 79
Waiting 60s to retry at index 95
Waiting 60s to retry at index 111
Waiting 60s to retry at index 127
Waiting 60s to retry at index 143
Waiting 60s to retry at index 159
Waiting 60s to retry at index 175
Waiting 60s to retry at index 191
Waiting 60s to retry at index 207
Waiting 60s to retry at index 223
Waiting 60s to retry at index 239
Waiting 60s to retry at index 255
Waiting 60s to retry at index 271
Waiting 60s to retry at index 287
Waiting 60s to retry at index 303
Waiting 60s to retry at index 319
Waiting 60s to retry at index 335
Waiting 60s to retry at index 351
Waiting 60s to retry at index 367
Waiting 60s to retry at index 383
Waiting 60s to retry at index 399
Waiting 60s to retry at index 415
Waiting 60s to retry at index 431
Waiting 60s to retry at index 447
Waiting 60s to retry at index 463
Waiting 60s to retry

In [ ]:
# export the dataframe to a new csv file
results_df.to_csv('data/salary_labelled_test_set_gemini_finetune.csv', index=False)

# Metrics

In [ ]:
preds = pd.read_csv('data/salary_labelled_test_set_gemini_finetune.csv')
test_df = pd.read_csv('data/salary_labelled_test_set_cleaned.csv')

In [ ]:
def extract_salary_format(text):
    """
    Extracts a JSON salary block from text and formats it as 'min-max-currency-frequency'
    """
    try:
        # Extract JSON block after ```json or at end of text
        match = re.search(r'\{.*?\}', text, re.DOTALL)

        salary_data = json.loads(match.group())

        min_salary = salary_data.get("MinSalary", 0)
        max_salary = salary_data.get("MaxSalary", 0)
        currency = salary_data.get("Currency", "None").lower()
        frequency = salary_data.get("Frequency", "None").lower()

        salary_text = f"{min_salary}-{max_salary}-{currency}-{frequency}".lower()
        salary_info = [salary_text, min_salary, max_salary, currency, frequency]

        return salary_info
    except Exception as e:
        # print("Error parsing salary block:", e)
        raise Exception("No JSON block found")

In [ ]:
preds_formatted = pd.DataFrame(columns=["y_pred", "y_pred_min", "y_pred_max", "y_pred_currency", "y_pred_frequency"])

In [ ]:
test_formatted = pd.DataFrame(columns=["y_true", "y_true_min", "y_true_max", "y_true_currency", "y_true_frequency"])

In [ ]:
for i in range(len(preds)):
  try:
    # post process prediction
    preds_formatted.loc[len(preds_formatted)] = extract_salary_format(preds.loc[i, 'y_pred'])

    # post process test
    salary_test_value = test_df.iloc[i].y_true
    salary_fields = salary_test_value.split("-")

    min_salary = salary_fields[0]
    max_salary = salary_fields[1]
    currency = salary_fields[2].lower()
    frequency = salary_fields[3].lower()
    salary_text = f"{min_salary}-{max_salary}-{currency}-{frequency}".lower()
    salary_info = [salary_text, min_salary, max_salary, currency, frequency]

    test_formatted.loc[len(test_formatted)] = salary_info
  except:
    print(f"issue at index {i}")

In [ ]:
preds_formatted

,y_pred,y_pred_min,y_pred_max,y_pred_currency,y_pred_frequency
0,1500-1800-myr-monthly,1500,1800,myr,monthly
1,60-60-hkd-hourly,60,60,hkd,hourly
2,0-0-none-none,0,0,none,none
3,0-0-none-none,0,0,none,none
4,0-0-none-none,0,0,none,none
...,...,...,...,...,...
562,26-26-nzd-hourly,26,26,nzd,hourly
563,0-0-none-none,0,0,none,none
564,0-0-none-none,0,0,none,none
565,1500-2500-myr-monthly,1500,2500,myr,monthly


In [ ]:
test_formatted

,y_true,y_true_min,y_true_max,y_true_currency,y_true_frequency
0,1500-1800-myr-monthly,1500,1800,myr,monthly
1,60-60-hkd-hourly,60,60,hkd,hourly
2,0-0-none-none,0,0,none,none
3,0-0-none-none,0,0,none,none
4,0-0-none-none,0,0,none,none
...,...,...,...,...,...
562,26-26-nzd-hourly,26,26,nzd,hourly
563,0-0-none-none,0,0,none,none
564,0-0-none-none,0,0,none,none
565,1500-2500-myr-monthly,1500,2500,myr,monthly


In [ ]:
preds_formatted.to_csv('salary_formatted_gemini_finetune.csv', index=False)

In [ ]:
acc_overall = (preds_formatted['y_pred'] == test_formatted['y_true']).mean() * 100
acc_min = (preds_formatted['y_pred_min'] == test_formatted['y_true_min']).mean() * 100
acc_max = (preds_formatted['y_pred_max'] == test_formatted['y_true_max']).mean() * 100
acc_curr = (preds_formatted['y_pred_currency'] == test_formatted['y_true_currency']).mean() * 100
acc_freq = (preds_formatted['y_pred_frequency'] == test_formatted['y_true_frequency']).mean() * 100

In [ ]:
pd.DataFrame(
    {
        'Overall': acc_overall,
        'Min Salary': acc_min,
        'Max Salary': acc_max,
        'Currency': acc_curr,
        'Frequency': acc_freq
    },
    index=['Accuracy (%)']
)

,Overall,Min Salary,Max Salary,Currency,Frequency
Accuracy (%),91.887125,94.885362,94.532628,98.412698,97.707231
